<a href="https://colab.research.google.com/github/karun-j/ci-cd-sandbox/blob/main/Generalized_Agents_Colabs/App_Integration_-_MT/Gemini_Apps_MT_ID_154/Agent-Gemini_Apps_MT_ID_154-Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sample ID**: Gemini_Apps_MT_ID_154


**Query**:  Can you please reply to Amy?


**DB Type**: Base Case


**Case Description**: Initially there exist contacts with the names Amy Marie and Amy Adams, and message that user got her message, and will plan a birthday surprise for James has not been sent to Amy Marie. Finally, the message that user got her message, and will plan a birthday surprise for James is expected to be sent to Amy Marie
```
<multiturn info>
Contact full name: Amy Marie (Ambiguity Resolution)
Send message: Send a message to Amy Marie that user got her message, and will plan a birthday surprise for James (Information Gathering)
</multiturn info>
```

**Global/Context Variables:**

**APIs:**
- contacts
- messages

**Databases:**

# Set Up

## Download relevant files

In [ ]:
import io
import os
import sys
import zipfile
import shutil
import re
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

# Version to download
VERSION = "0.1.4"  # This will be replaced dynamically

# Define paths
CONTENT_DIR = '/content'
APIS_DIR = os.path.join(CONTENT_DIR, 'APIs')
DBS_DIR = os.path.join(CONTENT_DIR, 'DBs')
SCRIPTS_DIR = os.path.join(CONTENT_DIR, 'Scripts')
FC_DIR = os.path.join(CONTENT_DIR, 'Schemas')
ZIP_PATH = os.path.join(CONTENT_DIR, f'APIs_V{VERSION}.zip')

# Google Drive Folder ID where versioned APIs zip files are stored
APIS_FOLDER_ID = '1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4'

# List of items to extract from the zip file
ITEMS_TO_EXTRACT = ['APIs/', 'DBs/', 'Scripts/', 'Schemas/']

# Clean up existing directories and files
for path in [APIS_DIR, DBS_DIR, SCRIPTS_DIR, FC_DIR, ZIP_PATH]:
    if os.path.exists(path):
        if os.path.isdir(path):
            shutil.rmtree(path)
        else:
            os.remove(path)

# Authenticate and create the drive service
auth.authenticate_user()
drive_service = build('drive', 'v3')

# Helper function to download a file from Google Drive
def download_drive_file(service, file_id, output_path, file_name=None, show_progress=True):
    """Downloads a file from Google Drive"""
    destination = output_path
    request = service.files().get_media(fileId=file_id)
    with io.FileIO(destination, 'wb') as fh:
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            if show_progress:
                print(f"Download progress: {int(status.progress() * 100)}%")

# 1. List files in the specified APIs folder
print(f"Searching for APIs zip file with version {VERSION} in folder: {APIS_FOLDER_ID}...")
apis_file_id = None

try:
    query = f"'{APIS_FOLDER_ID}' in parents and trashed=false"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    files = results.get('files', [])
    for file in files:
        file_name = file.get('name', '')
        if file_name.lower() == f'apis_v{VERSION.lower()}.zip':
            apis_file_id = file.get('id')
            print(f"Found matching file: {file_name} (ID: {apis_file_id})")
            break

except Exception as e:
    print(f"An error occurred while listing files in Google Drive: {e}")

if not apis_file_id:
    print(f"Error: Could not find APIs zip file with version {VERSION} in the specified folder.")
    sys.exit("Required APIs zip file not found.")

# 2. Download the found APIs zip file
print(f"Downloading APIs zip file with ID: {apis_file_id}...")
download_drive_file(drive_service, apis_file_id, ZIP_PATH, file_name=f'APIs_V{VERSION}.zip')

# 3. Extract specific items from the zip file to /content
print(f"Extracting specific items from {ZIP_PATH} to {CONTENT_DIR}...")
try:
    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_contents = zip_ref.namelist()

        for member in zip_contents:
            extracted = False
            for item_prefix in ITEMS_TO_EXTRACT:
                if member == item_prefix or member.startswith(item_prefix):
                    zip_ref.extract(member, CONTENT_DIR)
                    extracted = True
                    break

except zipfile.BadZipFile:
    print(f"Error: The downloaded file at {ZIP_PATH} is not a valid zip file.")
    sys.exit("Invalid zip file downloaded.")
except Exception as e:
    print(f"An error occurred during extraction: {e}")
    sys.exit("Extraction failed.")

# 4. Clean up
if os.path.exists(ZIP_PATH):
    os.remove(ZIP_PATH)

# 5. Add APIs to path
if os.path.exists(APIS_DIR):
    sys.path.append(APIS_DIR)
else:
    print(f"Error: APIS directory not found at {APIS_DIR} after extraction. Cannot add to path.")

# 6. Quick verification
# Check for the presence of the extracted items
verification_paths = [APIS_DIR, DBS_DIR, SCRIPTS_DIR]
all_present = True
print("\nVerifying extracted items:")
for path in verification_paths:
    if os.path.exists(path):
        print(f"✅ {path} is present.")
    else:
        print(f"❌ {path} is MISSING!")
        all_present = False

if all_present:
    print(f"\n✅ Setup complete! Required items extracted to {CONTENT_DIR}.")
else:
    print("\n❌ Setup failed! Not all required items were extracted.")
os.chdir(CONTENT_DIR)

Searching for APIs zip file with version 0.1.3 in folder: 1QpkAZxXhVFzIbm8qPGPRP1YqXEvJ4uD4...
Found matching file: APIs_V0.1.3.zip (ID: 1F6agaMHV1HaEz2pYX5G2GLTXPvjrivQP)
Download progress: 100%
Extracting specific items from /content/APIs_V0.1.3.zip to /content...

Verifying extracted items:
✅ /content/APIs is present.
✅ /content/DBs is present.
✅ /content/Scripts is present.

✅ Setup complete! Required items extracted to /content.

Generating FC Schemas
✅ Successfully generated 68 FC Schemas to /content/Schemas


## Install Dependencies and Clone Repositories

In [ ]:
!pip install -r /content/APIs/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 48.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 77.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.6/343.6 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.1/245.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.9/168.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.3/187.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.8 MB/s eta 0:0

## Import APIs and initiate DBs

In [ ]:
# proto_ignore
import random
import sys
import uuid

def patch_uuid4_deterministic(seed=42):
    rng = random.Random(seed)

    def deterministic_uuid4():
        return uuid.UUID(int=rng.getrandbits(128))

    sys.modules['uuid'].uuid4 = deterministic_uuid4

patch_uuid4_deterministic()


In [ ]:
import contacts
import messages

# Context variables
message_text = "Got your message, sure, let's plan a birthday surprise for James"

# Load default DBs
contacts.SimulationEngine.db.load_state("/content/DBs/ContactsDefaultDB.json")
messages.SimulationEngine.db.load_state("/content/DBs/MessagesDefaultDB.json")

# Local Variables
contact_given_names = ["Amy", "Amy"]
contact_family_names = ["Marie", "Adams"]

# Other Variables
contact_numbers = ["+919835757656", "+919545456637"]

# Add Contacts
for i in range(len(contact_given_names)):
    contacts.create_contact(
        given_name=contact_given_names[i],
        family_name=contact_family_names[i],
        phone=contact_numbers[i]
    )

# Initial Assertion
1. Assert that there exist contacts with names "Amy Marie" and "Amy Adams"
2. Assert that the message with the expected keywords has not been sent to Amy Marie

In [ ]:
import contacts
import messages
from Scripts.assertions_utils import *

# Local Variables
contact_names = ["Amy Marie", "Amy Adams"]
contact_name_confirmed = "Amy Marie"
expected_message_keywords = ['message', 'birthday', 'surprise', 'james']

all_contacts = contacts.list_contacts().get("contacts", [])
all_contact_names = []
for contact in all_contacts:
  names = contact.get("names", [])
  for name in names:
    all_contact_names.append(f"{name.get('givenName', '')} {name.get('familyName', '')}")

# 1. Assert that there exist contacts with names "Amy Marie" and "Amy Adams"
for contact_name in contact_names:
  contact_exists = compare_is_list_subset(contact_name, all_contact_names)
  assertion_message = f"Assertion 1 Failed: Contact '{contact_name}' was not found."
  assert contact_exists, assertion_message

# 2. Assert that the message with the expected keywords has not been sent to Amy Marie
all_messages = messages.utils._list_messages()
message_sent = any(
    compare_strings(message.get("recipient", {}).get("contact_name", ""), contact_name_confirmed) and
    all(compare_is_string_subset(keyword, message.get("message_body", "")) for keyword in expected_message_keywords)
    for message in all_messages
)

assertion_message = f"Assertion 2 Failed: Message with the keywords {expected_message_keywords} has already been sent to {contact_name_confirmed}."
assert not message_sent, assertion_message

# Action

**Simulated User**: Can you please reply to Amy?


In [ ]:
# proto_ignore
import contacts
import messages

In [ ]:
# Search contacts with name Amy
contacts.list_contacts("Amy")

{'contacts': [{'resourceName': 'people/c451b4cf3-6123-fdf7-7656-af7229d4beef',
   'etag': 'b02b61c4a3d70628ece66fa2fd5166e6',
   'names': [{'givenName': 'Amy', 'familyName': 'Marie'}],
   'emailAddresses': None,
   'phoneNumbers': [{'value': '+919835757656',
     'type': 'mobile',
     'primary': True}],
   'organizations': None,
   'isWorkspaceUser': None,
   'notes': None,
   'whatsapp': {'jid': '919835757656@s.whatsapp.net',
    'name_in_address_book': 'Amy Marie',
    'profile_name': 'Amy Marie',
    'phone_number': '+919835757656',
    'is_whatsapp_user': True},
   'phone': {'contact_id': 'people/c451b4cf3-6123-fdf7-7656-af7229d4beef',
    'contact_name': 'Amy Marie',
    'recipient_type': 'CONTACT',
    'contact_photo_url': None,
    'contact_endpoints': [{'endpoint_type': 'PHONE_NUMBER',
      'endpoint_value': '+919835757656',
      'endpoint_label': 'mobile'}]}},
  {'resourceName': 'people/c5304317f-af42-e12f-3838-b3268e944239',
   'etag': '0e51f30dc6a7ee39c4b032ccd7c524a5',
 

**Action Agent**: There are 2 contacts with the first name Amy, first is Amy Marie and second is Amy Adams, which of these contacts do you want to reply to?

**Simulated User**: I want to reply to Amy Marie

**Action Agent**: Sure, what message do you want to send to her?

**Simulated User**: Send the message to her that I got your message, and will plan a birthday surprise for James

In [ ]:
# Send message to Amy Marie
messages.send_chat_message(
    recipient={
            "contact_name": "Amy Marie",
            "contact_endpoints": [
                {
                    "endpoint_type": "PHONE_NUMBER",
                    "endpoint_value": "+919835757656",
                    "endpoint_label": "mobile"
                }
            ]
        },
    message_body="Got your message, will plan a birthday surprise for James")

{'status': 'success',
 'sent_message_id': 'msg_3',
 'emitted_action_count': 1,
 'action_card_content_passthrough': None}

**Action Agent**: Sent the message to Amy Marie. Is there anything else needed?

**Simulated User**: No, thanks

# Final Assertion
1. Assert that the message with the expected keywords has been sent to Amy Marie

In [ ]:
import contacts
import messages
from Scripts.assertions_utils import *

# Local Variables
contact_name_confirmed = "Amy Marie"
expected_message_keywords = ['message', 'birthday', 'surprise', 'james']

# 1. Assert that the message with the expected keywords has been sent to Amy Marie
all_messages = messages.utils._list_messages()
message_sent = any(
    compare_strings(message.get("recipient", {}).get("contact_name", ""), contact_name_confirmed) and
    all(compare_is_string_subset(keyword, message.get("message_body", "")) for keyword in expected_message_keywords)
    for message in all_messages
)

assertion_message = f"Assertion 1 Failed: Message with the keywords {expected_message_keywords} has not been sent to {contact_name_confirmed}."
assert message_sent, assertion_message